# Installing libraries

In [ ]:
#pip install pandas

# Importing libraries

In [3]:
import pandas as pd
import numpy as np

# Loading data files

In [4]:
#Full path for Principal and Subsidiary status blocks
path_11_12_ps = r"G:\Shared drives\Just Transition\Management\Datasets\Employment Unemployment 2011-12\Extracted files\Block_5_1_Usual principal activity particulars of household members.dta"
path_11_12_ss = r"G:\Shared drives\Just Transition\Management\Datasets\Employment Unemployment 2011-12\Extracted files\Block_5_2_Usual subsidiary economic activity particulars of household members.dta"

In [5]:
#Reading stata files from the stored path
df_11_12_ps = pd.read_stata(path_11_12_ps)
df_11_12_ss = pd.read_stata(path_11_12_ss)

In [6]:
#Creating unique Person ID to merge PS and SS blocks
list_person_ID = ["Sub_Round", "Sector", "FSU_Serial_No", "Hamlet_Group_Sub_Block_No", 
                  "Second_Stage_Stratum_No", "Sample_Hhld_No", "Person_Serial_No"]          # List of columns to concatenate for unique key. 
df_11_12_ps['person_ID'] = df_11_12_ps[list_person_ID].astype(str).agg("".join,axis=1)      # Unique key for PS block 
df_11_12_ss['person_ID'] = df_11_12_ss[list_person_ID].astype(str).agg("".join,axis=1)      # Unique key for SS block 

In [7]:
#Merging PS and SS blocks
df_11_12_combined = pd.merge(df_11_12_ps,df_11_12_ss,how = 'outer', on = 'person_ID', 
                             suffixes=('_PS', '_SS'), indicator=True, validate="1:1") # Merging PS and SS dataframes

In [8]:
#Correcting name of columns
df_11_12_combined = df_11_12_combined.rename(columns = {'Usual_SubsidiaryActivity_NIC2004': 'Usual_SubsidiaryActivity_NIC2008'})

In [9]:
#Displaying head of merged dataframe
df_11_12_combined.head()

,Round_Centre_Code_PS,FSU_Serial_No_PS,Round_PS,Sch_No_PS,Sample_PS,Sector_PS,State_Region_PS,District_PS,Stratum_PS,Sub_Stratum_No_PS,...,NSC_SS,MLT_SS,NSS_SR_SS,NSC_SR_SS,MLT_SR_SS,State_SS,District_Code,HHID_SS,Multiplier_comb_SS,_merge
0,005,70191,68,100,1,1,333,29,29,04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,006,70191,68,100,1,1,333,29,29,04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,007,70191,68,100,1,1,333,29,29,04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,006,70191,68,100,1,1,333,29,29,04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,007,70191,68,100,1,1,333,29,29,04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


# Dataformat changes

In [10]:
df_11_12_combined[['Usual_Principal_Activity_Status','Usual_Subsidiary_Activity_Status']] = df_11_12_combined[['Usual_Principal_Activity_Status','Usual_Subsidiary_Activity_Status']].apply(pd.to_numeric)

# New columns

In [11]:
#Usual status code
df_11_12_combined['usual_status_code'] = df_11_12_combined.apply(lambda row: row['Usual_Subsidiary_Activity_Status'] 
                                                                 if ((row['Usual_Principal_Activity_Status']>51) & (row['Whether_in_Subsidiary_Activity'] == '1')) 
                                                                 else row['Usual_Principal_Activity_Status'], 
                                                                 axis = 1 
                                                                 )

In [12]:
#Usual industry code

##Creating the variable
df_11_12_combined['usual_status_ind'] = df_11_12_combined.apply(lambda row:row['Usual_SubsidiaryActivity_NIC2008']
                                                                if ((row['Usual_Principal_Activity_Status']>51) & (row['Whether_in_Subsidiary_Activity'] == '1'))
                                                                else row['Usual_Principal_Activity_NIC2008'],
                                                                axis = 1
                                                                  )

##Storing only the first two digits
df_11_12_combined['usual_status_ind'] = df_11_12_combined['usual_status_ind'].astype(str).str[:2]

##Converting dtype to numeric
df_11_12_combined['usual_status_ind'] = pd.to_numeric(df_11_12_combined['usual_status_ind'])

In [14]:
#Categorising NIC codes into industry categories

##Codes based classification
conditions = [(df_11_12_combined["usual_status_ind"]>=1) & (df_11_12_combined["usual_status_ind"]<=3),
              (df_11_12_combined["usual_status_ind"]>=5) & (df_11_12_combined["usual_status_ind"]<=9),
              (df_11_12_combined["usual_status_ind"]>=10) & (df_11_12_combined["usual_status_ind"]<=33),
              (df_11_12_combined["usual_status_ind"]>=35) & (df_11_12_combined["usual_status_ind"]<=39),
              (df_11_12_combined["usual_status_ind"]>=41) & (df_11_12_combined["usual_status_ind"]<=43),
              (df_11_12_combined["usual_status_ind"]>=45) & (df_11_12_combined["usual_status_ind"]<=47),
              (df_11_12_combined["usual_status_ind"]>=49) & (df_11_12_combined["usual_status_ind"]<=53),
              (df_11_12_combined["usual_status_ind"]>=55) & (df_11_12_combined["usual_status_ind"]<=56),
              (df_11_12_combined["usual_status_ind"]>=58) & (df_11_12_combined["usual_status_ind"]<=99),
              ]

##Categories
choices = ["Agriculture", "Mining and quarrying", "Manufacturing", "Electricity and water supply", "Construction", "Trade", "Transport", "Accomodation and food services", "Other services"]

##Assigning categories based on conditions
df_11_12_combined["usual_status_ind_category"] = np.select(conditions, choices, default = None)

# Analysis

In [15]:
#Defining tabulate function

def tabulate (table, column):
    grouped_table = table.groupby(by = column, dropna = False)                          #Group table by selected column
    agg_table = grouped_table.agg(
        population_size = ("Multiplier_comb_PS", "sum"),                                #Popn size for each category in selected column
        sample_size = ("Multiplier_comb_PS", "size")                                    #Sample size for each category in selected column
    )
    popn_sum = agg_table[pd.notnull(agg_table.index)]['population_size'].sum()          #Summing non-null values
    agg_table['population_percentage'] = agg_table['population_size']*(100/popn_sum)     #Calculating popn shares. Ignore for NA
    agg_table['population_percentage'] = agg_table['population_percentage'].round(2)    #Round percentage to 2 digits
    agg_table = agg_table.sort_values(by = 'population_percentage',ascending = False)   #Sort table
    return agg_table

In [18]:
#Tabulating industry category
tabulate(df_11_12_combined,'usual_status_ind_category')

,population_size,sample_size,population_percentage
usual_status_ind_category,,,
NaN,6.678046e+08,284718,158.83
Agriculture,2.056013e+08,62042,48.90
Manufacturing,5.299371e+07,21502,12.60
Other services,4.970531e+07,31042,11.82
Construction,4.455395e+07,20291,10.60
Trade,3.916976e+07,22746,9.32
Transport,1.708703e+07,8908,4.06
Accomodation and food services,6.897312e+06,3543,1.64
Mining and quarrying,2.270032e+06,941,0.54


In [19]:
#Tabulating 2 digit NIC
tabulate(df_11_12_combined, 'usual_status_ind').sort_values(by = 'usual_status_ind')

,population_size,sample_size,population_percentage
usual_status_ind,,,
1.0,2.036178e+08,61140,48.43
2.0,5.935915e+05,360,0.14
3.0,1.389934e+06,542,0.33
5.0,5.215175e+05,362,0.12
6.0,1.521316e+05,41,0.04
...,...,...,...
95.0,2.489091e+06,1187,0.59
96.0,5.450906e+06,2858,1.30
97.0,3.470292e+06,1561,0.83


# Saving

In [23]:
#Exporting df_11_12_combined as pickle file

##Base path
export_base_path = r"G:\Shared drives\Just Transition\Management\Datasets\Employment Unemployment 2011-12\Modified data files"

#Saving
df_11_12_combined.to_pickle(export_base_path + r'\Block_5_1 & 5_2.pkl')
